In [0]:
import requests
from pyspark.sql import DataFrame

In [0]:
with requests.get('https://datahub.io/core/glacier-mass-balance/r/glaciers.csv', stream=True) as r:
  with open('/dbfs/glacier.csv', 'wb') as f:
    for chunk in r.iter_content(chunk_size=8192):
      f.write(chunk)

In [0]:
def get_data(url:str):
  filename = url.split('/')[-1]
  with requests.get('https://datahub.io/core/glacier-mass-balance/r/glaciers.csv', stream=True) as r:
    with open("/dbfs/{}".format(filename), 'wb') as f:
      for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)
  return filename

In [0]:
file_name = get_data('https://datahub.io/core/glacier-mass-balance/r/glaciers.csv')
file_name

Out[33]: 'glaciers.csv'

In [0]:
spark.read.format("csv").option("header","true").load("file:/dbfs/glacier.csv")

Out[34]: DataFrame[Year: string, Mean cumulative mass balance: string, Number of observations: string]

In [0]:
file_format = file_name.split(".")[-1]
file_format

Out[35]: 'csv'

In [0]:
def read_data(file_name):
  if file_format == 'csv':
    df = spark.read.format(file_format).option("header","true").load("file:/dbfs/{}".format(file_name))
  elif file_format == 'json':
    try:
      df = spark.read.format(file_format).load("file:/dbfs/{}".format(file_name))
    except:
      df = spark.read.format(file_format).option("multiline","true").load("file:/dbfs/{}".format(file_name))
  elif file_format == 'parquet':
    df = spark,read.format(file_format).load("file:/dbfs/{}".format(file_name))
  elif file_format == 'txt':
    df = spark.read.text("file:/dbfs/{}".format(file_name))
  return df

In [0]:
df = read_data(file_name)
display(df)

Year,Mean cumulative mass balance,Number of observations
1956,0,null
1957,-0.094,12
1958,-0.963,12
1959,-1.431,13
1960,-2.008,15
1961,-2.445,16
1962,-2.648,20
1963,-3,23
1964,-2.682,23
1965,-2.524,25


In [0]:
df.createOrReplaceTempView("df")

In [0]:
%sql
select * from df

Year,Mean cumulative mass balance,Number of observations
1956,0,null
1957,-0.094,12
1958,-0.963,12
1959,-1.431,13
1960,-2.008,15
1961,-2.445,16
1962,-2.648,20
1963,-3,23
1964,-2.682,23
1965,-2.524,25


In [0]:
%sql
create or replace temp view nintys as select * from df where Year like '19%' order by Year asc;
create or replace temp view modern as select * from df where Year like '20%' order by Year asc;

In [0]:
nintys_df = spark.sql("select * from nintys")
modern_df = spark.sql("select * from modern")

In [0]:

def transform_data(df: DataFrame):
  spark.sql("create or replace temp view nintys as select * from df where Year like '19%' order by Year asc;")
  nintys_df = spark.sql("select * from nintys")
  spark.sql("create or replace temp view modern as select * from df where Year like '20%' order by Year asc;")
  modern_df = spark.sql("select * from modern")
  return nintys_df, modern_df

In [0]:
x, y = transform_data(df)

In [0]:
display(y)

Year,Mean cumulative mass balance,Number of observations
2000,-12.91,57
2001,-13.181,57
2002,-13.609,57
2003,-14.134,57
2004,-14.866,55
2005,-15.683,58
2006,-16.398,58
2007,-16.938,57
2008,-17.313,58
2009,-17.765,58


In [0]:
display(x)

Year,Mean cumulative mass balance,Number of observations
1956,0,null
1957,-0.094,12
1958,-0.963,12
1959,-1.431,13
1960,-2.008,15
1961,-2.445,16
1962,-2.648,20
1963,-3,23
1964,-2.682,23
1965,-2.524,25


In [0]:
display(nintys_df)

Year,Mean cumulative mass balance,Number of observations
1956,0,null
1957,-0.094,12
1958,-0.963,12
1959,-1.431,13
1960,-2.008,15
1961,-2.445,16
1962,-2.648,20
1963,-3,23
1964,-2.682,23
1965,-2.524,25


In [0]:
nintys_df.take??

Signature: def take(num: int) -> List[Row]
Source:   
    def take(self, num: int) -> List[Row]:
        """Returns the first ``num`` rows as a :class:`list` of :class:`Row`.

        .. versionadded:: 1.3.0

        .. versionchanged:: 3.4.0
            Support Spark Connect.

        Parameters
        ----------
        num : int
            Number of records to return. Will return this number of records
            or all records if the DataFrame contains less than this number of records..

        Returns
        -------
        list
            List of rows

        Examples
        --------
        >>> df = spark.createDataFrame(
        ...     [(14, "Tom"), (23, "Alice"), (16, "Bob")], ["age", "name"])

        Return the first 2 rows of the :class:`DataFrame`.

        >>> df.take(2)
        [Row(age=14, name='Tom'), Row(age=23, name='Alice')]
        """
        return self.limit(num).collect()
File:      /databricks/spark/python/pyspark/sql/dataframe.py
Line:      1325
Type

In [0]:
display??

Signature: display(input=None, *args, **kwargs)
Source:   
    def display(self, input=None, *args, **kwargs):
        """
        Display plots or data.

        Display plot:
         - display() # no-op
         - display(matplotlib.figure.Figure)

        Display dataset:
         - display(spark.DataFrame)
         - display(list) # if list can be converted to DataFrame, e.g., list of named tuples
         - display(pandas.DataFrame)
         - display(koalas.DataFrame)
         - display(pyspark.pandas.DataFrame)

        Display any other value that has a _repr_html_() method

        For Spark 2.0 and 2.1:
         - display(DataFrame, streamName='optional', trigger=optional pyspark.sql.streaming.Trigger,
                   checkpointLocation='optional')

        For Spark 2.2+:
         - display(DataFrame, streamName='optional', trigger=optional interval like '1 second',
                   checkpointLocation='optional')
        """

        if input is None:
            # no-

In [0]:
display(modern_df)

Year,Mean cumulative mass balance,Number of observations
2000,-12.91,57
2001,-13.181,57
2002,-13.609,57
2003,-14.134,57
2004,-14.866,55
2005,-15.683,58
2006,-16.398,58
2007,-16.938,57
2008,-17.313,58
2009,-17.765,58


In [0]:
nintys_file_namez = spark.sql("(select * from nintys order by Year ASC limit 1) union (select * from nintys order by Year DESC limit 1)")
modern_file_namez = spark.sql("(select * from modern order by Year ASC limit 1) union (select * from modern order by Year DESC limit 1)")

In [0]:
display(nintys_file_namez)

Year,Mean cumulative mass balance,Number of observations
1956,0,null
1999,-12.551,57


In [0]:
display(modern_file_namez)

Year,Mean cumulative mass balance,Number of observations
2000,-12.91,57
2023,-29.738,53


In [0]:
modern_file_namez_df = modern_file_namez.collect()

In [0]:
modern_file_namez_df[0].__getitem__('Year') + "-" + modern_file_namez_df[1].__getitem__('Year')

Out[54]: '2000-2023'

In [0]:
def create_file_names():
  nintys_file_namez = spark.sql("(select * from nintys order by Year ASC limit 1) union (select * from nintys order by Year DESC limit 1)")
  modern_file_namez = spark.sql("(select * from modern order by Year ASC limit 1) union (select * from modern order by Year DESC limit 1)")
  nintys_file_namez_df = nintys_file_namez.collect()
  modern_file_namez_df = modern_file_namez.collect()
  nintys_file_name = nintys_file_namez_df[0].__getitem__('Year') + "-" + nintys_file_namez_df[1].__getitem__('Year') 
  modern_file_name = modern_file_namez_df[0].__getitem__('Year') + "-" + modern_file_namez_df[1].__getitem__('Year') 
  return nintys_file_name, modern_file_name

In [0]:
m,n = create_file_names()

In [0]:
print(m, n)

1956-1999 2000-2023


In [0]:
nintys_df.write.format('parquet').save("/dbfs/nintys_df1.parquet")

In [0]:
def write_df(file_type: str,dfs, file_names):
  for x,y in zip(dfs,file_names):
    m = x.write.format(file_type).save("/dbfs/{}.{}".format(y, file_type))
  return m 
  

As the below files were already existing, so deleting these files first and running the command again

In [0]:
dbutils.fs.rm("dbfs:/dbfs/1956-1999.parquet", recurse=True)
dbutils.fs.rm("dbfs:/dbfs/2000-2023.parquet", recurse=True)

Out[64]: True

In [0]:
write_df("parquet",[x, y],[m, n])

In [0]:
dbutils.fs.ls('/dbfs/')